# Tarea 1: Piping
### Pablo Correa e Ian Gross

## Librerías

In [10]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte

import fluids.units as units


## Problema 1
En un sistema de refrigeración, el refrigerante R134a fluye a través de un tubo de cobre estándar tipo
L de 3/4 de pulgada. En este sistema, se utiliza un desuperheter para calentar agua, como se muestra en
la Figura P4.3. El R134a ingresa al desuperheter a 200 psia y 225 ºF. El desuperheter extrae suficiente
energía, QDSH, de manera que el R134a ingresa al condensador como un vapor saturado a una presión
levemente inferior. En el condensador, se transfiere energía desde el refrigerante a una tasa de 60000
Btu/h. Esto hace que el R134a salga del condensador como un líquido saturado con una caída de presión
despreciable.

a. Determine la pérdida de carga en el desuperheater.

In [15]:
g = cte.value(u'standard acceleration of gravity')

# Transform inches to meter
D = 3/4 * 0.0254 #Pulgadas a metros

z = 1

material = 'Copper'
epsilon = fld.material_roughness(material)

fluid = "R134a"
rho = cp.PropsSI('D','T', 380, 'P',1.379e6, fluid)
mu = cp.PropsSI('V','T', 380, 'P',1.379e6, fluid)
print(rho, mu)

50.80505814934574 1.5294536955578792e-05


b. Determine la velocidad (m/s) del R134a al ingresar al condensador.

c. Determine la velocidad (m/s) del R134a al salir del condensador.

d. Investigue sobre correlaciones de pérdida de carga en tuberías con flujo bigfásico. Proponga
una expresión para estimar la pérdida de carga en el condensador. ¿qué información sería necesaria?

e. Cuál es la diferencia entre una tubería tipo L y una tipo M

Las tuberías tipo L tienen un espesor proporcional mayor al de la tuberías tipo M. Esto les permite soportar mayores presiones y, por lo tanto, se utilizan en aplicaciones industriales o de transporte de gas licuado o gas natural. A motivo del mayor requerimiento de material, las tuberías L tienen un costo más elevado. Además, el mayor espesor de la tubería L le da una durabilidad notablemente mayor frente al mismo uso.

[1] https://woolfplumbing.com.au/blog/the-differences-between-copper-pipes-type-l-and-type-m

[2] https://sodimac.falabella.com/sodimac-cl/page/como-elegir-tuberias-y-fittings-de-cobre

## Problema 2
Gas Natural fluye con una velocidad promedio de 100 ft/s por una tubería de cobre de 300 m y $1\frac{1}{4}$
inch de diámetro. La temperatura del gas es 20 ◦C y la presión inicial es de 1 psig.

a. Asumiendo que el flujo es incompresible, determine la variación de la presión (pérdida de
carga) en la tubería.

b. Determine el grado de error introducido por el supuesto.

c. Verifique la validez del supuesto.

d. Haga una evaluación (gráfico) del error vs longitud de la tubería.

e. ¿Cuánto influencia la temperatura en la pérdida de carga? Grafique

## Problema 3
Se utiliza una tubería horizontal para transportar etanol (alcohol etílico) a 20 ◦C a lo largo de una
distancia de 3 km. La tubería está hecha de acero comercial. El caudal requerido de etanol es de 40
L/s. La bomba conectada a esta tubería puede superar una caída de presión de hasta 200 kPa en la
tubería.

a. Especifique el tamaño apropiado (diámetro nominal) de una tubería de tipo Schedule 40 para esta tubería.

b. Grafique el caudal máximo que sería entregado por diferentes diametros del schedule
indicado.

## Problema 4
Considere el plano hidráulico de la localidad de Hualañé.

a. Establezca un código computacional, que permita estimar la pérdida de carga máxima en el sistema, cuando todas las casas tienen una llave de
lavamanos abierta.

b. Grafique la pérdida de carga asociada a cada casa si eldiametro de todas las
tuberías es de 1 pulgada.